In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
df_2022 = pd.read_csv('../data_csv/clean_2022_df.csv')
df_2022.head()

,Unnamed: 0,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,0,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2022,purchase-only,1,295.78,299.33
1,1,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2022,purchase-only,2,301.74,303.24
2,2,traditional,monthly,USA or Census Division,East South Central Division,DV_ESC,2022,purchase-only,1,347.85,348.10
3,3,traditional,monthly,USA or Census Division,East South Central Division,DV_ESC,2022,purchase-only,2,353.88,353.26
4,4,traditional,monthly,USA or Census Division,Middle Atlantic Division,DV_MA,2022,purchase-only,1,319.52,319.19


In [2]:
# Drop the non-beneficial ID columns
df_2022 = df_2022.drop(columns = ['Unnamed: 0'], axis= 1)
df_2022.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2022,purchase-only,1,295.78,299.33
1,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2022,purchase-only,2,301.74,303.24
2,traditional,monthly,USA or Census Division,East South Central Division,DV_ESC,2022,purchase-only,1,347.85,348.10
3,traditional,monthly,USA or Census Division,East South Central Division,DV_ESC,2022,purchase-only,2,353.88,353.26
4,traditional,monthly,USA or Census Division,Middle Atlantic Division,DV_MA,2022,purchase-only,1,319.52,319.19


In [3]:
#check for bucketing
# Generate our categorical variable list
cat_2022 = df_2022.dtypes[df_2022.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
df_2022[cat_2022].nunique()

hpi_type       1
hpi_flavor     1
frequency      1
level         10
place_name    10
yr             1
dtype: int64

In [4]:
#11 uniques values needs to be checked 
# Check the unique value counts to see if binning is required
df_2022.level.value_counts()

East North Central Division    2
East South Central Division    2
Middle Atlantic Division       2
Mountain Division              2
New England Division           2
Pacific Division               2
South Atlantic Division        2
West North Central Division    2
West South Central Division    2
United States                  2
Name: level, dtype: int64

In [5]:
#there are a substantial number of datapoints. leave it alone no need to bucket
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_2022[cat_2022]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat_2022)
encode_df.head()

C:\Users\renee\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,hpi_type_traditional,hpi_flavor_monthly,frequency_USA or Census Division,level_East North Central Division,level_East South Central Division,level_Middle Atlantic Division,level_Mountain Division,level_New England Division,level_Pacific Division,level_South Atlantic Division,...,place_name_DV_ESC,place_name_DV_MA,place_name_DV_MT,place_name_DV_NE,place_name_DV_PAC,place_name_DV_SA,place_name_DV_WNC,place_name_DV_WSC,place_name_USA,yr_purchase-only
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
df_2022 = df_2022.merge(encode_df,left_index=True, right_index=True)
df_2022 = df_2022.drop(cat_2022,1)
df_2022.head()

C:\Users\renee\AppData\Local\Temp\ipykernel_27848\62941678.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_2022 = df_2022.drop(cat_2022,1)


,place_id,period,index_nsa,index_sa,hpi_type_traditional,hpi_flavor_monthly,frequency_USA or Census Division,level_East North Central Division,level_East South Central Division,level_Middle Atlantic Division,...,place_name_DV_ESC,place_name_DV_MA,place_name_DV_MT,place_name_DV_NE,place_name_DV_PAC,place_name_DV_SA,place_name_DV_WNC,place_name_DV_WSC,place_name_USA,yr_purchase-only
0,2022,1,295.78,299.33,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2022,2,301.74,303.24,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2022,1,347.85,348.10,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2022,2,353.88,353.26,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2022,1,319.52,319.19,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
# Remove loan status target from features data
y = df_2022.hpi_type_traditional
X = df_2022.drop(columns=["index_sa","index_nsa"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 1.000


In [9]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
1/1 [==============================] - 1s 663ms/step - loss: 1.0246 - accuracy: 0.0667
Epoch 2/50
1/1 [==============================] - 0s 5ms/step - loss: 1.0017 - accuracy: 0.0667
Epoch 3/50
1/1 [==============================] - 0s 5ms/step - loss: 0.9796 - accuracy: 0.0667
Epoch 4/50
1/1 [==============================] - 0s 6ms/step - loss: 0.9582 - accuracy: 0.1333
Epoch 5/50
1/1 [==============================] - 0s 5ms/step - loss: 0.9374 - accuracy: 0.2000
Epoch 6/50
1/1 [==============================] - 0s 5ms/step - loss: 0.9178 - accuracy: 0.2667
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: 0.8990 - accuracy: 0.3333
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: 0.8810 - accuracy: 0.3333
Epoch 9/50
1/1 [==============================] - 0s 5ms/step - loss: 0.8636 - accuracy: 0.3333
Epoch 10/50
1/1 [==============================] - 0s 5ms/step - loss: 0.8476 - accuracy: 0.3333
Epoch 11/50
1/1 [====================